In [1]:
import numpy as _np
import spinmob as _s
import matplotlib.pyplot as _plt
import os as _os
from scipy.optimize import curve_fit

In [2]:
data_file_names = []
betas = []
beta_start = "beta_"
beta_end = "_par"
for name in _os.listdir(_os.getcwd()):
    if name.find("beta")>=0:
        data_file_names.append(name)
        betas.append(float(name[name.find(beta_start)+len(beta_start):name.find(beta_end)]))
print(data_file_names)
print(betas)


['beta_0.75_parallel(1).csv', 'beta_0.75_parallel.csv', 'beta_0.775_parallel(1).csv', 'beta_0.775_parallel.csv', 'beta_0.825_parallel(1).csv', 'beta_0.825_parallel.csv', 'beta_0.85_parallel(1).csv', 'beta_0.85_parallel.csv', 'beta_0.875_parallel(1).csv', 'beta_0.875_parallel.csv', 'beta_0.8_parallel(1).csv', 'beta_0.8_parallel.csv', 'beta_0.925_parallel(1).csv', 'beta_0.925_parallel.csv', 'beta_0.95_parallel(1).csv', 'beta_0.95_parallel.csv', 'beta_0.975_parallel(1).csv', 'beta_0.975_parallel.csv', 'beta_0.9_parallel(1).csv', 'beta_0.9_parallel.csv', 'beta_1.025_parallel(1).csv', 'beta_1.025_parallel.csv', 'beta_1.05_parallel(1).csv', 'beta_1.05_parallel.csv', 'beta_1.075_parallel(1).csv', 'beta_1.075_parallel.csv', 'beta_1.0_parallel(1).csv', 'beta_1.0_parallel.csv', 'beta_1.1_parallel(1).csv', 'beta_1.1_parallel.csv']
[0.75, 0.75, 0.775, 0.775, 0.825, 0.825, 0.85, 0.85, 0.875, 0.875, 0.8, 0.8, 0.925, 0.925, 0.95, 0.95, 0.975, 0.975, 0.9, 0.9, 1.025, 1.025, 1.05, 1.05, 1.075, 1.075, 1

In [3]:
data_boxes = _s.data.load_multiple(paths = data_file_names)

In [4]:
len(data_boxes)

30

In [5]:
def exp_decay(x, gamma, amp):
    return amp*_np.exp(-x/gamma)

def correlation(data, data_average):
    data_correlations = []
    correlation_distance = _np.array(range(len(data)-1))+1
    for distance in correlation_distance:
        correlations = data[:-distance]*data[distance::]-data_average**2
        data_correlations.append(_np.average(correlations))
    return data_correlations, correlation_distance

def fit_exp_decay(data, distacnes, p0):
    popt, cov = curve_fit(exp_decay, distacnes[:2000], data[:2000], p0=p0)
    return [popt, cov]

def plot_exp_decay_fit(correlation, distances, popt, ep_str, n_or_e):
    _plt.plot(distances, correlation, label="data")
    _plt.axhline(0, linestyle="--", color="grey", alpha=0.5)
    _plt.plot(distances[:2000], exp_decay(distances[:2000], *popt), label="fit")
    _plt.title(n_or_e+" correlation for ε="+ep_str, size=35)
    _plt.ylabel(r'$\tilde{e_n}\tilde{e}_{n+d} - \langle \tilde{e}\rangle^2$', size=35)
    _plt.xlabel("d", size=35)
    _plt.tick_params(axis="both", labelsize=30)
    _plt.legend(fontsize=30)
    _plt.show()

def choose_unc(es, es_popt, ns, ns_popt):
    es_uncs = []
    ns_uncs = []
    num_trials = len(es)
    for position, block in enumerate(block_sizes):
        blocked_es = _np.reshape(es, (int(num_trials/block), block))
        block_avg_es = _np.average(blocked_es, axis=1)
        unc = _np.std(block_avg_es)/_np.sqrt(len(block_avg_es))
        es_uncs.append(unc)

        blocked_ns = _np.reshape(ns, (int(num_trials/block), block))
        block_avg_ns = _np.average(blocked_ns, axis=1)
        unc = _np.std(block_avg_ns)/_np.sqrt(len(block_avg_ns))
        ns_uncs.append(unc)

        print("E Unc: ", es_uncs[-1], "E correlation/unc: ", _np.sqrt(exp_decay(block, *es_popt))/unc)
        print("n Unc: ", ns_uncs[-1], "n correlation/unc: ", _np.sqrt(exp_decay(block, *ns_popt))/unc)
        print(" num blocks: ", len(block_avg_es), " blocksize: ", block, "choice number", position)
        print("\r\r\r")
    which_block = int(input("Which block_size?"))
    return [block_sizes[which_block], es_uncs[which_block], ns_uncs[which_block]]


In [6]:
chis0 = data_boxes[12][0][1000:]
chis0_average = _np.average(chis0)
chis0_correlation, chis0_distances = correlation(chis0, chis0_average)
p0 = [200, 30]
chis0_popt, chis0_cov = fit_exp_decay(chis0_correlation, chis0_distances, p0)
p0 = [50, 800]
chis0_popt, chis0_cov = fit_exp_decay(chis0_correlation, chis0_distances, p0)
plot_exp_decay_fit(chis0_correlation, chis0_distances, chis0_popt, "", r"$\chi$")
chis0_popt

array([ 11.96853298, 968.08069006])

In [7]:
block_size = 125
exp_decay(block_size, *chis0_popt)/chis0_average*100

0.004307919698703232

In [8]:
def calcualte_windings_avg(data_box):
    return _np.average(data_box[2])
def calcualte_windings_unc(data_box):
    return _np.std(data_box[2])/_np.sqrt(len(data_box[2])/block_size)

windings_avg = [calcualte_windings_avg(data_box) for data_box in data_boxes]
windings_unc = [calcualte_windings_unc(data_box) for data_box in data_boxes]

In [9]:
betas_final = []
chis = []
chis_unc = []
N=20
for beta in set(betas):
    betas_final.append(beta)
    indexes = _np.nonzero(_np.array(betas)==beta)
    avgs = _np.array(windings_avg)[indexes]
    uncs = _np.array(windings_unc)[indexes]
    avgs = avgs[_np.nonzero(uncs)]
    uncs = uncs[_np.nonzero(uncs)]
    weights = uncs**-2
    chis.append(_np.sum(weights*avgs)/_np.sum(weights))
    chis_unc.append(1/_np.sqrt(_np.sum(weights)))
sort_indexes = _np.argsort(betas_final)
betas_final = _np.array(betas_final)[sort_indexes]
chis = _np.array(chis)[sort_indexes]
chis_unc = _np.array(chis_unc)[sort_indexes]
print(betas_final)
print(chis)
print(chis_unc)

[0.75  0.775 0.8   0.825 0.85  0.875 0.9   0.925 0.95  0.975 1.    1.025
 1.05  1.075 1.1  ]
[4.12000000e-03 1.91200632e-03 8.38720608e-03 1.78155810e-02
 5.42701118e-02 1.11668225e-01 3.06592832e-01 5.16909847e-01
 9.35776680e-01 1.20703205e+00 1.53867416e+00 1.78652296e+00
 2.01751739e+00 2.25667793e+00 2.47930020e+00]
[0.00226468 0.00180541 0.00314657 0.00474025 0.00857137 0.01291878
 0.0235927  0.03281468 0.05004449 0.06294235 0.07618843 0.08969996
 0.10117627 0.11074738 0.12285391]


In [10]:
_plt.errorbar(betas_final, chis, yerr=chis_unc, marker=".", color="blue", zorder=1)
_plt.title(r"$\chi_\omega \: vs \: \beta \: at \: N=20, \: \epsilon=0.01$", size=18)
_plt.xlabel(r"$\beta$", size=18)
_plt.ylabel(r"$\chi_\omega$", size=18)
_plt.legend()

No handles with labels found to put in legend.


In [11]:
chis_unc/chis*100

array([54.96799598, 94.42488076, 37.516294  , 26.60732319, 15.79390449,
       11.56889627,  7.69512487,  6.34824015,  5.34790965,  5.2146382 ,
        4.95156347,  5.02092424,  5.01488951,  4.90754014,  4.95518506])

In [12]:
for jj in range(len(chis)):
    print(betas_final[jj], chis[jj], chis_unc[jj])

0.75 0.00412 0.002264681434551006
0.775 0.001912006321490562 0.0018054096892704218
0.8 0.00838720608052631 0.0031465688919556007
0.825 0.017815580962482924 0.004740249204437283
0.85 0.05427011183033771 0.008571369630605234
0.875 0.11166822459615378 0.012918781067399672
0.9 0.30659283188035263 0.023592701265924724
0.925 0.5169098470292232 0.03281467842691488
0.95 0.9357766803016367 0.05004449136966139
0.975 1.2070320504675711 0.06294235438385021
1.0 1.5386741561593251 0.07618842747371263
1.025 1.7865229573588368 0.08969996425442045
1.05 2.0175173894353673 0.10117626794278058
1.075 2.256677930311366 0.1107473751545197
1.1 2.4793002040418837 0.1228539133345776


In [24]:
for data_box in data_boxes:
    print(len(data_box[0]))

100000
5000
100000
5000
100000
5000
100000
5000
100000
5000
100000
5000
100000
5000
100000
5000
100000
5000
100000
5000
100000
5000
100000
5000
100000
5000
100000
5000
100000
5000
